In [3]:
import json

import pandas as pd
import pickle

In [2]:
from utils.eval_utils import *

In [37]:
def get_pckl_keys(dir_path):
    """

    Function to show the keys in the h5py file.

    """

    for scale in os.listdir(dir_path):
        for location in os.listdir(os.path.join(dir_path, scale)):
            print(location)
            if location.endswith(".pkl"):
                with open(os.path.join(dir_path, scale, location), 'rb') as f:
                    data = pickle.load(f)
                    print(f"Horizons: {data.keys()}")
                    print(f"Seasons: {data[list(data.keys())[0]].keys()}")
                    print(f"Models: {data[1]['Winter'][0].keys()}")

                    return data
                    

In [39]:
dict_result_n_ahead = get_pckl_keys("/Users/nikolaushouben/Desktop/WattCast/data/evaluations")

building_1.pkl
Horizons: dict_keys([1, 4, 8, 24, 48])
Seasons: dict_keys(['Winter', 'Summer'])
Models: dict_keys(['RandomForest', 'XGBModel', 'LightGBMModel', 'BlockRNNModel', 'NBEATSModel', 'TFTModel', 'LinearRegressionModel'])


In [147]:
from utils.pipeline import Config, derive_config_params

with open("init_config.json", "r") as f:
    init_config = json.load(f)


config = Config().from_dict(init_config)

config = derive_config_params(config)



In [168]:
model = "BlockRNNModel"
season = "Winter"
horizon = 4

metrics_list = [metrics_dict[metric] for metric in config.metrics]

keys = ["horizon", "season", "model"]

df_metrics = pd.DataFrame(columns = keys + config.metrics)

for horizon in dict_result_n_ahead.keys():
    for season in dict_result_n_ahead[horizon].keys():
        gt = dict_result_n_ahead[horizon][season][2]
        persistence = dict_result_n_ahead[horizon][season][1][f"{config.hours_persistence}-Hour Persistence"] 
        perf_dict_persistence = calc_error_scores(metrics_list, persistence, gt)
        df_persistence = pd.DataFrame.from_dict(perf_dict_persistence, orient='index').T
        df_persistence["horizon"] = 0
        df_persistence["season"] = season
        df_persistence["model"] = f"{config.hours_persistence}-Hour Persistence"
        df_metrics = pd.concat([df_metrics, df_persistence], axis=0)

        for model in dict_result_n_ahead[horizon][season][0].keys():
            df_metrics_model = pd.DataFrame(columns = keys + config.metrics)

            for pred_batch in dict_result_n_ahead[horizon][season][0][model]:

                pred, gt_sliced = make_index_same(pred_batch, gt)

                perf_dict = calc_error_scores(metrics_list, pred, gt_sliced)

                df_perf = pd.DataFrame.from_dict(perf_dict, orient='index').T

                df_metrics_model = pd.concat([df_metrics_model, df_perf], axis=0)

                # special treatment for max_peak_error and mean_n_peak_error
                if "max_peak_error" in df_metrics_model.columns:
                    df_metrics_model["max_peak_error"] = df_metrics_model["max_peak_error"].max()
                if "mean_n_peak_error" in df_metrics_model.columns:
                    df_metrics_model["mean_n_peak_error"] = df_metrics_model["mean_n_peak_error"].sort_values(ascending=False)[::horizon][:5].mean()
        
            
            df_summary = df_metrics_model.mean().to_frame().T
            # add keys
            df_summary["horizon"] = horizon
            df_summary["season"] = season
            df_summary["model"] = model
            # add to df_metrics
            df_metrics = pd.concat([df_metrics, df_summary], axis=0)
        rmse_persistence = df_metrics[df_metrics["model"] == f"{config.hours_persistence}-Hour Persistence"]["rmse"]
        df_metrics["rmse_skill"] = 1 - df_metrics["rmse"] / rmse_persistence.values[0]
        df_metrics = df_metrics[~(df_metrics["model"] == f"{config.hours_persistence}-Hour Persistence")]


In [170]:
df_metrics.to_csv("data/evaluations/5_building/df_metrics_building_1.csv")